In [1]:
import ollama

In [2]:
modelfile = """
FROM llama3.1
SYSTEM Sort the input list of numbers in ascending order. The list is comma-separated. Only return the list of numbers, not the input prompt.
"""

ollama.create(model="sortLlama", modelfile=modelfile)

{'status': 'success'}

In [3]:
def sortLlama(list):
    list_str = ",".join([str(i) for i in list])
    response = ollama.chat(
        model="sortLlama",
        messages=[
            {
                "role": "user",
                "content": list_str,
            },
        ],
    )
    list_response = response["message"]["content"]

    return [int(i) for i in list_response.split(",")]

sortLlama([3, 2, 1])

[1, 2, 3]